In [1]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style
import subprocess

In [8]:
with open('./res/lab2.json') as lab2_file:
    data = json.load(lab2_file)
rpn = data['rpn']
tables = data['tables']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

In [9]:
res = ''
for i in rpn:
    res += str(i) + ' '
res

'n 15 = b 30 = c 35 = m n 2 Ф 1 НА summ 0 = a b c + = i 5 = НЦ M1: i 5 > M2 УПЛ m i 2 Ф 1 = summ summ m i 2 Ф + = КЦ M1 БП M2: '

In [15]:
subprocess.run(["python3", "reverse-polish-notation.py"])
with open('./res/lab2.json') as lab2_file:
    data = json.load(lab2_file)
rpn = data['rpn']
tables = data['tables']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

result = ''
stack = []
vars_declaring = []

function_context = None
while_context = None
if_m = None

for item in rpn:
    print(item)
    
    splitted = item.split(' ')
    if len(splitted) == 4:
        proc = 'КЦ'
    elif len(splitted) == 3:
        proc = 'ELSE'
        arity = splitted[0]
    else:
        [arity, proc,] = [0, item] if len(splitted) < 2 else splitted
    print(arity, '|', proc, stack)
#     print()
    if proc == 'НП':
        result += 'function '
        function_context = 'func'
    elif arity == 'НЦ':
        result += 'while ('
        while_context = 'while'
    elif proc == 'УПЛ' and while_context:
        result += str(stack.pop()) + ') {\n'
        while_context = None
    elif proc == 'УПЛ' and not while_context:
        result += 'if (' + str(stack.pop()) + ') {\n'
        if_m = str(arity) + ':'
    elif proc == 'ELSE':
        result += '} else {\n'
        if_m = str(arity) + ':'
    elif proc == if_m:
        result += '}\n'
    elif proc in ['КП', 'КЦ']:
        result += '}\n'
    elif proc in operations and proc != '=':
        b = stack.pop()
        a = stack.pop()
        stack.append(str(a) + ' ' + proc + ' ' + str(b))
    elif proc == 'НА':
        for i in range(int(arity)):
            var = stack.pop().split('[')
            result += var[0] + ' = new Array(' + var[1].split(']')[0] + ');\n'
    elif proc == '=':
        exp = stack.pop()
        result += str(stack.pop()) + ' = ' + str(exp) + ';\n'
    elif proc == 'Ф':
        args = ''
        for i in range(int(arity) - 1):
            args = stack.pop() + ', ' + args
        args = args[:-2]
        stack.append(str(stack.pop()) + '(' + args + ')')
        if function_context:
            result += str(stack.pop()) + ' {\n'
            function_context = None
    elif proc == 'АЭМ':
        args = ''
        for i in range(int(arity) - 1):
            args = stack.pop() + ', ' + args
        args = args[:-2]
        stack.append(str(stack.pop()) + '[' + args + ']')
    elif proc == 'RETURN':
        result += 'return ' + str(stack.pop()) + ';\n'
    else:
        print('\"' + str(proc) + '" <> "' + str(if_m) + '"')
        stack.append(proc)

print(stack)
print()
print(result)

n
0 | n []
"n" <> "None"
15
0 | 15 ['n']
"15" <> "None"
=
0 | = ['n', '15']
b
0 | b []
"b" <> "None"
30
0 | 30 ['b']
"30" <> "None"
=
0 | = ['b', '30']
c
0 | c []
"c" <> "None"
35
0 | 35 ['c']
"35" <> "None"
=
0 | = ['c', '35']
m
0 | m []
"m" <> "None"
n
0 | n ['m']
"n" <> "None"
2 АЭМ
2 | АЭМ ['m', 'n']
1 НА
1 | НА ['m[n]']
summ
0 | summ []
"summ" <> "None"
0
0 | 0 ['summ']
"0" <> "None"
=
0 | = ['summ', '0']
a
0 | a []
"a" <> "None"
b
0 | b ['a']
"b" <> "None"
c
0 | c ['a', 'b']
"c" <> "None"
+
0 | + ['a', 'b', 'c']
=
0 | = ['a', 'b + c']
i
0 | i []
"i" <> "None"
5
0 | 5 ['i']
"5" <> "None"
=
0 | = ['i', '5']
a
0 | a []
"a" <> "None"
a
0 | a ['a']
"a" <> "None"
>
0 | > ['a', 'a']
M1 УПЛ
M1 | УПЛ ['a > a']
a
0 | a []
"a" <> "M1:"
5
0 | 5 ['a']
"5" <> "M1:"
=
0 | = ['a', '5']
b
0 | b []
"b" <> "M1:"
1
0 | 1 ['b']
"1" <> "M1:"
=
0 | = ['b', '1']
M2 БП M1:
M2 | ELSE []
b
0 | b []
"b" <> "M2:"
a
0 | a ['b']
"a" <> "M2:"
=
0 | = ['b', 'a']
M2:
0 | M2: []
НЦ M3:
НЦ | M3: []
i
0 | i []
"i" <